# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [11]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2019-05-12	Question answering for spoken lecture processing	ICASSP	Spoken Question Answering with deep learning methods	Ünlü, Merve, Ebru Arısoy and Murat Saraçlar. (2019). "Question answering for spoken lecture processing" <i>ICASSP</i>	qa-for-spoken-lecture-processing	http://academicpages.github.io/files/UnluArisoySaraclar2019ICASSP.pdf
2019-04-26	A Question Answering System for Spoken Lectures in Turkish	SIU	Spoken Question Answering with deep learning methods on Turkish QA dataset.	Ünlü, Merve, Ebru Arısoy and Murat Saraçlar. (2019). "A Question Answering System for Spoken Lectures in Turkish" <i>SIU</i>	spoken-qa-turkish	http://academicpages.github.io/files/UnluArisoySaraclar2019SIU.pdf




## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [12]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2019-05-12,Question answering for spoken lecture processing,ICASSP,Spoken Question Answering with deep learning m...,"Ünlü, Merve, Ebru Arısoy and Murat Saraçlar. (...",qa-for-spoken-lecture-processing,http://academicpages.github.io/files/UnluAriso...
1,2019-04-26,A Question Answering System for Spoken Lecture...,SIU,Spoken Question Answering with deep learning m...,"Ünlü, Merve, Ebru Arısoy and Murat Saraçlar. (...",spoken-qa-turkish,http://academicpages.github.io/files/UnluAriso...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [13]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [14]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

2019-04-26-spoken-qa-turkish.md  2019-05-12-qa-for-spoken-lecture-processing.md


In [16]:
!cat ../_publications/2019-04-26-spoken-qa-turkish.md

---
title: "A Question Answering System for Spoken Lectures in Turkish"
collection: publications
permalink: /publication/2019-04-26-spoken-qa-turkish
excerpt: 'Spoken Question Answering with deep learning methods on Turkish QA dataset.'
date: 2019-04-26
venue: 'SIU'
paperurl: 'http://academicpages.github.io/files/UnluArisoySaraclar2019SIU.pdf'
citation: 'Ünlü, Merve, Ebru Arısoy and Murat Saraçlar. (2019). &quot;A Question Answering System for Spoken Lectures in Turkish&quot; <i>SIU</i>'
---
Spoken Question Answering with deep learning methods on Turkish QA dataset.

[Download paper here](http://academicpages.github.io/files/UnluArisoySaraclar2019SIU.pdf)

Recommended citation: Ünlü, Merve, Ebru Arısoy and Murat Saraçlar. (2019). "A Question Answering System for Spoken Lectures in Turkish" <i>SIU</i>